In [1]:
import pandas as pd

In [ ]:
df=pd.read_csv(r"D:\uni\AdvancedTopics\Assignment3\DB\card_games\cards.csv")
df=df.sample(20,random_state=42)

In [3]:
df_rule=pd.read_csv(r"D:\uni\AdvancedTopics\Assignment3\DB\card_games\ruling.csv")

In [4]:
result = df_rule[df_rule['uuid'].isin(df['uuid'])]

In [8]:
df.to_csv("DB/card_games/cards_sample20.csv")
result.to_csv("DB/card_games/ruling_sample.csv")

In [1]:
from groq import Groq
import pandas as pd
import re
from typing import List, Dict, Union, Optional,Any
import json
from dotenv import load_dotenv
import os

load_dotenv()

groq_api_key=os.getenv('GROQ_API_KEY')
model_name=os.getenv('MODEL_NAME')
# Initialize Groq client
client = Groq(api_key=groq_api_key)

# ====================== GENERIC UDFs ======================

def analyze_sentiment(text: str) -> float:
    """Sentiment Analysis: Returns score between -1 (negative) and 1 (positive)"""
    if not text:
        return 0.0
    
    chat_completion = client.chat.completions.create(
        messages=[{
            "role": "system",
            "content": "Analyze sentiment of this text. Return ONLY a float between -1 (negative) and 1 (positive)."
        }, {
            "role": "user",
            "content": text
        }],
        model=model_name, 
        temperature=0.0,
        max_tokens=10
    )
    return float(chat_completion.choices[0].message.content)

def align_to_schema(data: Dict, domain: str) -> List[Dict]:
    prompt = f"""Convert this table column mapping to Schema.org/{domain} property names:
    For each column name in the input data, provide the most appropriate Schema.org property name.
    The input data is: {data}
    
    Return ONLY a valid JSON array where each element is an object with:
    - the original column name as key
    - the corresponding Schema.org property name as value
    
    Example output format:
    {{
      "name": "name",
      "flavorText": "description",
      "text": "text",
      "type": "additionalType"
    }}"""
    
    chat_completion = client.chat.completions.create(
        messages=[{
            "role": "user", 
            "content": prompt + "\n\nJSON output required:"
        }],
        model=model_name,
        temperature=0.0,
        response_format={"type": "json_object"}
    )
    
    # Get the response and convert it to a list of mappings
    result = json.loads(chat_completion.choices[0].message.content)
    return result

def classify_entity(text: str, classes: List[str]) -> str:
    """Entity Classification: Categorizes text into predefined classes"""
    prompt = f"""Classify this text into one of {classes}:
    {text}
    Return ONLY the class name."""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model_name,
        temperature=0.0,
        max_tokens=20
    )
    return chat_completion.choices[0].message.content.strip()

def summarize(text: str, max_length: int = 50) -> str:
    """Summarization: Condenses text to specified length"""
    prompt = f"""Summarize this in under {max_length} characters:
    {text}
    Summary:"""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model_name,
        temperature=0.0,
        max_tokens=max_length
    )
    return chat_completion.choices[0].message.content.strip()

def extract_entities(text: str, entity_types: List[str]) -> List[str]:
    prompt = f"""Extract ALL {entity_types} from this text:
    "{text}"
    Return ONLY a Python-style list like ["item1", "item2"] with no other text."""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model_name,
        temperature=0.0,
        max_tokens=100
    )
    
    try:
        return eval(chat_completion.choices[0].message.content)
    except:
        return []
    try:
        # Handle both direct string response and JSON object
        response = chat_completion.choices[0].message.content
        if isinstance(response, str):
            return eval(response)
        elif isinstance(response, dict):
            return list(response.values())[0]  # Assumes first value is the list
    except:
        return []  # Fallback to empty list on parsing failure

def impute_missing(
    row: Dict[str, Any],
    column_to_impute: str,
    context_columns: List[str],
    type_hint: Optional[str] = None,
    id_columns: Optional[List[str]] = None
) -> Dict[str, Any]:
    """
    Generic missing value imputation using specified context columns.
    
    Args:
        row: Input data row as dictionary
        column_to_impute: Key containing missing value
        context_columns: List of columns to use for context
        type_hint: Expected data type ("numeric", "date", "boolean", "text")
        id_columns: Columns to exclude from context (even if in context_columns)
    
    Returns:
        Dictionary with imputed value (or original if imputation fails)
    """
    result_row=row.copy()
    # Return early if no imputation needed
    if row.get(column_to_impute) not in [None, "NULL", "", "null"] and not pd.isna(row.get(column_to_impute)):
        return row[column_to_impute]
    
    # Filter context columns
    id_columns = id_columns or []
    available_context = [
        (k, str(row[k])) for k in context_columns 
        if k in row and k != column_to_impute 
        and row[k] is not None and k not in id_columns
    ]
    
    # Build context string
    context_str = "\n".join(f"{k}: {v}" for k, v in available_context)
    if not context_str:
        return row[column_to_impute]  # No context available
    
    # Determine type hint
    type_instruction = ""
    if type_hint:
        type_map = {
            "numeric": "Return only a number without symbols",
            "date": "Use YYYY-MM-DD format",
            "boolean": "Return 'true' or 'false'",
            "text": "Return a string without quotes"
        }
        type_instruction = f"\nIMPORTANT: {type_map.get(type_hint, '')}"
    
    prompt = f"""Impute missing value for '{column_to_impute}' based on this context:
    {context_str}
    {type_instruction}
    Return ONLY the raw value with no additional text or formatting."""
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": prompt}],
        model=model_name,
        temperature=0.0,
        max_tokens=50
    )
    
    result_row[column_to_impute] = chat_completion.choices[0].message.content.strip()
    return result_row


def augment_schema(row: Dict, new_field: str, prompt: str) -> Dict:
    """Schema Augmentation: Adds new derived field"""
    user_prompt = f"""{prompt}
    Data: {row}
    Return ONLY the value for {new_field}."""
    
    chat_completion = client.chat.completions.create(
        messages=[{"role": "user", "content": user_prompt}],
        model=model_name,
        temperature=0.0,
        max_tokens=50
    )
    row[new_field] = chat_completion.choices[0].message.content.strip()
    return row


In [2]:
# Load sample data
cards_df = pd.read_csv("DB/card_games/cards_sample20.csv")
rulings_df = pd.read_csv("DB/card_games/ruling_sample.csv")

In [ ]:
def test_sentiment_analysis_001(cards: pd.DataFrame) -> List[str]:
    """
    Test Case ID: entity_classification_001
    Calculates the sentiment score on cards' flavourtext and returns the negative ones
    "Which cards have flavor text with negative sentiment?"
    equivalent SQL:
    SELECT name
    FROM cards
    WHERE flavorText IS NOT NULL
    AND analyze_sentiment(flavorText) < 0;
    """
    negative_cards = []
    for _, card in cards.iterrows():
        if pd.notna(card['flavorText']):
            sentiment = analyze_sentiment(card['flavorText'])
            if sentiment < 0:  # Negative sentiment
                negative_cards.append(card['name'])
    return negative_cards

#print(test_sentiment_analysis_001(cards_df))

['Ostiary Thrull']


In [ ]:
def test_entity_classification_001(cards: pd.DataFrame) -> List[Dict]:
    """
    Test Case ID: entity_classification_001
    Classifies the archetype of 'Twilight Prophet' based on text using
    "What is the archetype of Twilight Prophet?"
    equivalent SQL:
    SELECT 
    'Twilight Prophet' AS card_name,
    classify_entity(
        text, 
        ARRAY['Aggro', 'Control', 'Combo']  -- Array of possible archetypes
    ) AS archetype
    FROM cards
    WHERE name = 'Twilight Prophet'
    LIMIT 1; 
    """
    # Get the specific card data
    card = cards[cards['name'] == 'Twilight Prophet'].iloc[0]
    archetype=classify_entity(card['text'],["Aggro","Control","Combo"])
    return archetype

#print(test_entity_classification_001(cards_df))

Control


In [ ]:
def test_entity_recognition_001(cards: pd.DataFrame) -> List[str]:
    """
    Test Case ID: entity_recognition_001
    Identifies people or creature entities in a text and returns all the cards that present identified entities in their flavour text
    Which cards mention creatures or people in their flavor text?
    equivalent SQL:
    SELECT DISTINCT name
    FROM cards
    WHERE flavorText IS NOT NULL
    AND extract_entities(flavorText, ARRAY['PERSON','CREATURE']::text[]) IS NOT NULL;
    
    """
    entity_cards = []
    for _, card in cards.iterrows():
        if pd.notna(card['flavorText']):
            entities = extract_entities(card['flavorText'], ["PERSON","CREATURE"])
            if entities:  # Found mentions of people/creatures
                entity_cards.append(card['name'])
    return entity_cards

#print(test_entity_recognition_001(cards_df))

['Crested Sunmare', 'Civic Wayfinder', 'Helm of the Gods', 'Ancestral Memories', 'Lady Sun']


In [ ]:
def test_schema_alignment_001(cards: pd.DataFrame) -> List[Dict]:
    """
    Test Case ID: schema_alignment_001  
    Map card properties to Schema.org types.
    equivalent SQL:
    SELECT 
        name,
        align_to_schema(
            JSON_BUILD_OBJECT(
                'name', name,
                'flavorText', flavorText,
                'text', text,
                'type', type
            ),
            'Game'
        ) AS mapped_properties
    FROM cards
    WHERE name = 'Lady Sun';
      
    """
    aligned = []
    for _, card in cards.iterrows():
        if card['name'] == "Lady Sun":  # From test case
            mapped = align_to_schema({
                "name": card['name'],
                "flavorText": card['flavorText'],
                "text": card['text'],
                "type": card['type']
            }, "Game")
            aligned.append({
                "name": card['name'],
                "mapped_properties": mapped
            })
    return aligned
#print(test_schema_alignment_001(cards_df))


[{'name': 'Lady Sun', 'mapped_properties': {'name': 'name', 'flavorText': 'description', 'text': 'gameplayDescription', 'type': 'gameItemVariant'}}]


In [ ]:
def test_imputation_missing_001(cards: pd.DataFrame) -> List[Dict]:
    """
    Test Case ID: imputation_missing_001
    Calls the llm to try and fill a specific column value based on external knowledge and the other columns values 
    "Fill in the missing toughness value of creatures based on other card fields."
    equivalent SQL:
    WITH creature_cards AS (
    SELECT *
    FROM cards
    WHERE LOWER(type) LIKE '%creature%'
    ),
    cards_to_impute AS (
    SELECT *
    FROM creature_cards
    WHERE toughness IS NULL 
        OR toughness = '' 
        OR LOWER(toughness) = 'null'
    )
    SELECT 
    name,
    impute_missing(
        JSON_BUILD_OBJECT(
        'name', name,
        'type', type,
        'power', power,
        'text', text
        ),
        'toughness',
        ARRAY['name', 'type', 'power', 'text'],
        'numeric'
    ) AS imputed_toughness
    FROM cards_to_impute;
    
    """
    # Get the creature type cards
    creature_cards = cards_df[cards_df['type'].str.contains('Creature', case=False, na=False)].copy()
    # List of cards to modify 
    cards_to_update = ["Civic Wayfinder", "Dream Stalker", "Esper Sojourners"]

    # Update toughness to None/NULL for those specific cards
    creature_cards.loc[creature_cards['name'].isin(cards_to_update), 'toughness'] = None
    imputed_cards=[]
    creature_cards=creature_cards.to_dict('records')
    for card in creature_cards:
        if pd.isna(card["toughness"]) or card["toughness"] in [None, "NULL", "", "null"]:
            imputed_cards.append({"name":card["name"],"imputed_toughness":impute_missing(
        row=card,
        column_to_impute="toughness",
        context_columns=["name", "type", "power", "text"],
        type_hint="numeric")["toughness"]})
    return(imputed_cards)
    
print(test_imputation_missing_001(cards_df))


[{'name': 'Esper Sojourners', 'imputed_toughness': '2'}, {'name': 'Dream Stalker', 'imputed_toughness': '2'}, {'name': 'Civic Wayfinder', 'imputed_toughness': '3'}]


In [ ]:
def test_summarization_001(cards: pd.DataFrame, rulings: pd.DataFrame) -> List[Dict]:
    """
    Test Case ID: summarization_001
    Calls the llm to summarize a text
    "Summarize rulings for 'Esper Sojourners'."
    equivalent SQL:
    WITH card_uuid AS (
    SELECT uuid
    FROM cards
    WHERE name = 'Esper Sojourners'
    ),
    combined_rulings AS (
    SELECT 
        c.name,
        STRING_AGG(r.text, E'\n' ORDER BY r.text) AS rulings_text
    FROM card_uuid cu
    JOIN cards c ON c.uuid = cu.uuid
    JOIN rulings r ON r.uuid = cu.uuid
    GROUP BY c.name
    )
    SELECT 
    name,
    summarize(rulings_text, 100) AS summary
    FROM combined_rulings;
    
    """
    summaries = []
    card_name = "Esper Sojourners"
    card_uuid = cards[cards['name'] == card_name]['uuid'].iloc[0]
    card_rulings = rulings[rulings['uuid'] == card_uuid]
    
    if not card_rulings.empty:
        combined_rulings = "\n".join(card_rulings['text'])
        summary = summarize(combined_rulings, 100)
        summaries.append({
            "name": card_name,
            "summary": summary
        })
    return summaries

#print(test_summarization_001(cards_df,rulings_df))

[{'name': 'Esper Sojourners', 'summary': 'Cycling is an activated ability, interacts with ability effects, not spell effects.'}]


In [ ]:
def test_classification_002(cards: pd.DataFrame, rulings: pd.DataFrame) -> List[str]:
    """
    Test Case ID: classification_002
    Identifies cards with rulings classified as "damage_prevention" 
    "Which cards have rulings that mention 'damage prevention'?"
    equivalent SQL:
    SELECT DISTINCT c.name
    FROM rulings r
    JOIN cards c ON r.uuid = c.uuid
    WHERE classify_entity(r.text, ARRAY['damage_prevention', 'other']) = 'damage_prevention';
    
    """
    damage_prevention_cards = []
    
    for _, ruling in rulings.iterrows():
        # Classify the ruling text
        classification = classify_entity(
            text=ruling['text'],
            classes=["damage_prevention", "other"],
        )
        
        if classification == "damage_prevention":
            card = cards[cards['uuid'] == ruling['uuid']].iloc[0]
            damage_prevention_cards.append(card['name'])
    
    return list(set(damage_prevention_cards))
#print(test_classification_002(cards_df,rulings_df))

['Twilight Prophet', 'Lady Sun', 'Esper Sojourners', 'Illusions of Grandeur', 'Helm of the Gods', 'Crested Sunmare', 'Blizzard Brawl']


In [ ]:
def test_entity_recognition_002(cards: pd.DataFrame) -> List[Dict]:
    """
    Test Case ID: entity_recognition_002
    call llm to analyze the text to identify cards that give a buff and count how many identified cards for each type
    How many cards of each type have text that inplies giving a buff?
    equivalent SQL:
    SELECT 
        type,
        COUNT(CASE WHEN extract_entities(text, ARRAY['buff', 'stat increase']) IS NOT NULL THEN 1 END) AS buff_count,
        COUNT(*) AS total_cards
    FROM cards
    WHERE text IS NOT NULL
    GROUP BY type;
    
    """
    results = []
    
    # Group by card type and process each group
    for card_type, group in cards.groupby('type'):
        buff_count = 0
        
        for text in group['text'].dropna():  # Skip NaN values
            extracted = extract_entities(text, ["buff", "stat increase"])
            if extracted:
                buff_count += 1
                
        
        results.append({
            "type": card_type,
            "buff_count": buff_count,
            "total_cards": len(group)
        })
    return results

#print(test_group_by_001(cards_df))

[{'type': 'Artifact Creature — Horse', 'buff_count': 1, 'total_cards': 1}, {'type': 'Artifact Creature — Vedalken Wizard', 'buff_count': 0, 'total_cards': 1}, {'type': 'Artifact — Equipment', 'buff_count': 1, 'total_cards': 1}, {'type': 'Creature — Elf Druid Warrior', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Horse', 'buff_count': 1, 'total_cards': 1}, {'type': 'Creature — Illusion', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Merfolk Soldier', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Thrull', 'buff_count': 0, 'total_cards': 1}, {'type': 'Creature — Vampire Cleric', 'buff_count': 0, 'total_cards': 1}, {'type': 'Enchantment', 'buff_count': 2, 'total_cards': 2}, {'type': 'Instant', 'buff_count': 1, 'total_cards': 2}, {'type': 'Land', 'buff_count': 1, 'total_cards': 3}, {'type': 'Legendary Creature — Human Advisor', 'buff_count': 0, 'total_cards': 1}, {'type': 'Snow Sorcery', 'buff_count': 1, 'total_cards': 1}, {'type': 'Sorcery', 'buff_count': 

In [ ]:
def test_analyze_sentiment_summarize_001(cards: pd.DataFrame, rulings: pd.DataFrame) -> List[Dict]:
    """
    Test Case ID: analyze_sentiment_summarize_001
    calls an llm to assing a sentiment score to flavour text of the cards,
    than takes the cards with positive score(>0.5)
    and finds those wid rulings, finally calls again the llm to produce
    a summary of said rulings
    "Cards with positive flavor text sentiment and complex rulings."
    equivalent SQL:
    SELECT
    c.name,
    summarize(string_agg(r.text, E'\n'), 100) AS summary,
    'positive' AS sentiment
    FROM cards c
    JOIN rulings r ON c.uuid = r.uuid
    WHERE c.flavorText IS NOT NULL
    AND analyze_sentiment(c.flavorText) > 0.5
    GROUP BY c.name;
    
    """
    complex_cards = []
    for _, card in cards.iterrows():
        if pd.notna(card['flavorText']):
            sentiment = analyze_sentiment(card['flavorText'])
            if sentiment > 0.5:
                card_rulings = rulings[rulings['uuid'] == card['uuid']]
                if not card_rulings.empty:
                    combined_rulings = "\n".join(card_rulings['text'])
                    summary = summarize(combined_rulings, 100)
                    complex_cards.append({
                        "name": card['name'],
                        "summary": summary,
                        "sentiment": "positive"
                    })
    return complex_cards
#print(test_analyze_sentiment_summarize_001(cards_df,rulings_df))

0.0
0.7
0.0
0.08
0.75
0.8
0.0
-0.5
0.0
0.0
0.75
[{'name': 'Crested Sunmare', 'summary': "Crested Sunmare's ability triggers if you gained life earlier in the turn, not during the end step.", 'sentiment': 'positive'}, {'name': 'Helm of the Gods', 'summary': "You control Aura spells targeting opponents' permanents.", 'sentiment': 'positive'}]


In [ ]:
def test_entity_impute_classify_001(cards: pd.DataFrame) -> List[Dict]:
    """
    Test Case ID: analyze_sentiment_summarize_001
    For creature cards with missing rarity:
    finds the cards with missing rarity  values, calls the llm to predict it and
    classify archetype
    "For creature cards with missing rarity, predict the rarity based on their stats and classify them by their gameplay archetype."
    equivalent SQL:
    WITH missing_rarity_creatures AS (
    SELECT *
    FROM cards
    WHERE LOWER(type) LIKE '%creature%'
        AND rarity IS NULL
    ),
    processed_cards AS (
    SELECT
        name,
        impute_missing(
        JSON_BUILD_OBJECT(
            'name', name,
            'type', type,
            'power', power,
            'toughness', toughness,
            'text', text,
            'manaCost', manaCost
        ),
        'rarity',
        ARRAY['name', 'type', 'power', 'toughness', 'text', 'manaCost'],
        'text'
        )->>'rarity' AS imputed_rarity,
        classify_entity(
        CONCAT(
            'Card: ', name, '\n',
            'Stats: ', power, '/', toughness, ' (MV: ', manaCost, ')\n',
            'Text: ', COALESCE(text, '')
        ),
        ARRAY['aggro', 'control', 'combo', 'midrange', 'hybrid']
        ) AS archetype
    FROM missing_rarity_creatures
    )
    SELECT
    name,
    imputed_rarity,
    LOWER(archetype) AS archetype
    FROM processed_cards;
    
    """
    results = []

    # Force rarity=None for our test cases
    cards.loc[cards['name'].isin([
        "Dream Stalker", 
        "Ostiary Thrull", 
        "Crested Sunmare"
    ]), 'rarity'] = None
    
    # Filter creatures with missing rarity
    creatures = cards[
        (cards['type'].str.contains('Creature', case=False, na=False)) & 
        (cards['rarity'].isna())
    ].copy()
    
    for _, card in creatures.iterrows():
        # Convert card data to dict for UDF processing
        card_data = card.to_dict()
        
        # Impute missing rarity
        imputed_card = impute_missing(
            row=card_data,
            column_to_impute="rarity",
            context_columns=["name", "type", "power", "toughness", "text", "manaCost"],
            type_hint="text"
        )
        
        #Classify archetype
        archetype = classify_entity(
            text=f"""
            Card: {card['name']}
            Stats: {card['power']}/{card['toughness']} (MV: {card['manaCost']})
            Text: {card['text']}
            """,
            classes=["aggro", "control", "combo", "midrange", "hybrid"],
        )
        
        results.append({
            "name": card['name'],
            "imputed_rarity": imputed_card["rarity"],
            "archetype": archetype.lower()
        })
    
    return results
#print(test_entity_impute_classify_001(cards_df))

[{'name': 'Crested Sunmare', 'imputed_rarity': 'Rare', 'archetype': 'midrange'}, {'name': 'Dream Stalker', 'imputed_rarity': 'Rare', 'archetype': 'control'}, {'name': 'Ostiary Thrull', 'imputed_rarity': 'Uncommon', 'archetype': 'control'}]
